In [1]:
import gmaps
from config import g_key
import pandas as pd 
import requests

In [2]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
city_data_gmaps_df = pd.read_csv('weather_data/cities.csv')
city_data_gmaps_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
gmaps.configure(api_key=g_key)

locations = city_data_gmaps_df[['Lat','Lng']]
maxTemp = city_data_gmaps_df['Max Temp']
adj_maxTemp_weights = [max(temp, 0) for temp in maxTemp]

fig = gmaps.figure(center = (30.0,31.0), zoom_level =1.5)

heatmap_layer = gmaps.heatmap_layer(locations, weights = adj_maxTemp_weights, 
                                    dissipating = False, max_intensity = 300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heatmap_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Percent Humidity Heatmap
locations = city_data_gmaps_df[['Lat','Lng']]
humidity = city_data_gmaps_df['Humidity']

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                            dissipating = False, max_intensity=300, point_radius=4)
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Percent Cloudiness Heatmap
locations = city_data_gmaps_df[['Lat','Lng']]
clouds = city_data_gmaps_df['Cloudiness']

heatmap_layer = gmaps.heatmap_layer(locations,weights=clouds,
                    dissipating=False, max_intensity=300, point_radius=4)
fig = gmaps.figure(center = (30.0,31.0), zoom_level =1.5)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Wind Speed Heatmap
locations = city_data_gmaps_df[['Lat','Lng']]
wind = city_data_gmaps_df['Wind Speed']

heatmap_layer = gmaps.heatmap_layer(locations, weights=wind,
                            dissipating = False, max_intensity=300, point_radius=4)
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# user prompts for input min and max temp range
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    

preferred_city_data_gmaps_df = city_data_gmaps_df.loc[(city_data_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                      (city_data_gmaps_df['Max Temp']>=min_prompt_temp)]
# dropna() for srop null values rows
clean_preferred_city_data_gmaps_df = preferred_city_data_gmaps_df.dropna()
clean_preferred_city_data_gmaps_df.count()

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


City_ID       176
City          176
Country       176
Date          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [10]:
hotel_df = clean_preferred_city_data_gmaps_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Carnarvon,AU,75.20,-24.87,113.63,
3,Hithadhoo,MV,85.30,-0.60,73.08,
9,Gudari,IN,86.29,19.35,83.78,
11,Grand Gaube,MU,78.80,-20.01,57.66,
13,Domoni,KM,84.00,-12.26,44.53,


In [ ]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params = {'key': g_key, 'type':'lodging', 'radius' : 5000}
    params['location'] = f"{lat},{lng}"
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

hotel_df.head()

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [ ]:
hotel_df.count()

In [ ]:
v_locations = hotel_df[['Lat','Lng']]
v_maxTemp = hotel_df['Max Temp']

fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(v_locations, weights= v_maxTemp, max_intensity = 300,\
                                 point_radius = 4, dissipating=False)
marker_layer = gmaps.marker_layer(v_locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

In [ ]:
v_locations = hotel_df[['Lat','Lng']]
v_maxTemp = hotel_df['Max Temp']
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>'''

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows() ]

fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(v_locations, weights= v_maxTemp, max_intensity = 300,\
                                 point_radius = 4, dissipating=False)
marker_layer = gmaps.marker_layer(v_locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig